In [0]:
%sql
CREATE TABLE departments (
    dept_id INT,
    dept_name STRING
);

INSERT INTO departments VALUES
(1, 'Engineering'),
(2, 'HR'),
(3, 'Sales'),
(4, 'Finance');

In [0]:
%sql
CREATE TABLE employees (
    emp_id INT,
    emp_name STRING,
    salary INT,
    dept_id INT,
    manager_id INT
);

INSERT INTO employees VALUES
(101, 'Alice', 120000, 1, NULL),   -- Head Engineering
(102, 'Bob', 80000, 1, 101),
(103, 'Charlie', 75000, 1, 101),
(104, 'David', 110000, 2, NULL),   -- Head HR
(105, 'Eva', 60000, 2, 104),
(106, 'Frank', 50000, 3, 108),     -- Reports to Hank
(107, 'Grace', 90000, NULL, 101),  -- No department
(108, 'Hank', 95000, 3, 101),
(109, 'Ivy', 75000, 3, 108),
(110, 'Jake', 75000, 3, 108),
(111, 'OrphanMgr', 70000, 4, 999); -- Invalid manager_id


In [0]:
%sql
CREATE TABLE employee_duplicates (
    emp_id INT,
    emp_name STRING
);

INSERT INTO employee_duplicates VALUES
(201, 'Tom'),
(202, 'Jerry'),
(201, 'Tom'),
(203, 'Anna'),
(202, 'Jerry');


In [0]:
%sql
CREATE TABLE customers (
    customer_id INT,
    customer_name STRING,
    city STRING
);

INSERT INTO customers VALUES
(1, 'John', 'New York'),
(2, 'Emma', 'Los Angeles'),
(3, 'Robert', 'Chicago'),
(4, 'Sophia', 'Houston'),
(5, 'Liam', 'Boston');  -- No orders


In [0]:
%sql
CREATE TABLE orders (
    order_id INT,
    customer_id INT,
    order_date DATE
);

INSERT INTO orders VALUES
(1001, 1, DATE('2024-01-01')),
(1002, 1, DATE('2024-01-05')),
(1003, 2, DATE('2024-02-10')),
(1004, 3, DATE('2024-02-15')),
(1005, 99, DATE('2024-03-01'));  -- Orphan customer


In [0]:
%sql
CREATE TABLE products (
    product_id INT,
    product_name STRING,
    category STRING
);

INSERT INTO products VALUES
(10, 'Laptop', 'Electronics'),
(11, 'Phone', 'Electronics'),
(12, 'Keyboard', 'Accessories'),
(13, 'Mouse', 'Accessories'),
(14, 'Tablet', 'Electronics');


In [0]:
%sql
CREATE TABLE order_items (
    order_id INT,
    product_id INT,
    quantity INT,
    unit_price INT
);

INSERT INTO order_items VALUES
(1001, 10, 2, 1000),
(1001, 11, 1, 800),
(1002, 10, 1, 1000),
(1003, 12, 5, 200),
(1004, 13, 3, 300);


In [0]:
%sql
CREATE TABLE students (
    student_id INT,
    student_name STRING
);

INSERT INTO students VALUES
(1, 'Alex'),
(2, 'Brian'),
(3, 'Cathy');


In [0]:
%sql
CREATE TABLE courses (
    course_id INT,
    course_name STRING
);

INSERT INTO courses VALUES
(101, 'Math'),
(102, 'Physics'),
(103, 'Chemistry');


In [0]:
%sql
CREATE TABLE enrollments (
    student_id INT,
    course_id INT
);

INSERT INTO enrollments VALUES
(1, 101),
(1, 102),
(2, 101),
(3, 103);


In [0]:
%sql
CREATE TABLE employees_old (
    emp_id INT,
    emp_name STRING,
    salary INT
);

INSERT INTO employees_old VALUES
(101, 'Alice', 115000),   -- Salary changed
(102, 'Bob', 80000),
(103, 'Charlie', 75000),
(112, 'NewHire', 50000);  -- Missing in current table


In [0]:
%sql
SHOW TABLES

#Find employees who don’t have departments

In [0]:
select * from employees E
left join departments D
on D.dept_id=E.dept_id
where D.dept_id is null


#Find common records between two tables


In [0]:
select * from employees E
join departments D
on D.dept_id=E.dept_id


#Find unmatched records from both tables


In [0]:
SELECT 
    e.emp_id AS current_emp_id,
    o.emp_id AS old_emp_id,
    e.emp_name AS current_name,
    o.emp_name AS old_name
FROM employees e
FULL OUTER JOIN employees_old o
    ON e.emp_id = o.emp_id
WHERE e.emp_id IS NULL 
   OR o.emp_id IS NULL;



#Self join: employees and their managers


In [0]:
select * from employees e
join employees e1
on e.emp_id<>e1.emp_id
and e.emp_id=e1.manager_id

In [0]:
select * from employees_old


In [0]:
select * from employees

In [0]:
select * from departments

In [0]:
select * from employees E
left join departments D
on D.dept_id=E.dept_id
where D.dept_id is null